In [13]:
%pip install seaborn
%pip install matplotlib
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline 


In [3]:
import tensorflow as tf

# Detect GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable dynamic memory growth for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs detected.")
    except RuntimeError as e:
        print("Error configuring GPU memory growth:", e)
else:
    print("No GPU detected by TensorFlow. Training will use CPU.")


No GPU detected by TensorFlow. Training will use CPU.


In [4]:
CLASSES = ["non demented" , "very mild demented" , "mild demented", "moderate demented"]
IMAGESIZE = 224

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up your ImageDataGenerator with normalization (rescaling)
traindata_gen = ImageDataGenerator(
    rescale=1.0/255.0  # Normalize pixel values to the range [0, 1]
)

# Load your training data
traindata = traindata_gen.flow_from_directory(
    directory="data/train",
    target_size=(224, 224),  # Resize images to the desired dimensions
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=CLASSES,  # Specify the class names
)

# Set up the test/dev data generator with the same normalization
testdata_gen = ImageDataGenerator(
    rescale=1.0/255.0
)

# Load your validation data
devdata = testdata_gen.flow_from_directory(
    directory="data/dev",
    target_size=(224, 224),  # Ensure consistent image size
    class_mode='categorical',  # Class mode should match the training setup
    classes=CLASSES,
)


Found 30467 images belonging to 4 classes.
Found 6528 images belonging to 4 classes.


In [6]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Load pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Prompt user for layers to unfreeze
def set_trainable_layers(model, layers_to_unfreeze):
    for i, layer in enumerate(model.layers):
        layer.trainable = i in layers_to_unfreeze
        status = "Trainable" if layer.trainable else "Frozen"
        print(f"Layer {i}: {layer.name} - {status}")

# Example: Unfreeze layers 15 to the last one
layers_to_unfreeze = list(range(15, len(base_model.layers)))
set_trainable_layers(base_model, layers_to_unfreeze)

# Build your model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduces spatial dimensions
    Dense(100, activation='relu'),  # Fully connected layer
    Dense(len(CLASSES), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Layer 0: input_layer_1 - Frozen
Layer 1: block1_conv1 - Frozen
Layer 2: block1_conv2 - Frozen
Layer 3: block1_pool - Frozen
Layer 4: block2_conv1 - Frozen
Layer 5: block2_conv2 - Frozen
Layer 6: block2_pool - Frozen
Layer 7: block3_conv1 - Frozen
Layer 8: block3_conv2 - Frozen
Layer 9: block3_conv3 - Frozen
Layer 10: block3_pool - Frozen
Layer 11: block4_conv1 - Frozen
Layer 12: block4_conv2 - Frozen
Layer 13: block4_conv3 - Frozen
Layer 14: block4_pool - Frozen
Layer 15: block5_conv1 - Trainable
Layer 16: block5_conv2 - Trainable
Layer 17: block5_conv3 - Trainable
Layer 18: block5_pool - Trainable


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        51,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,766,392 (56.33 MB)

 Trainable params: 7,131,128 (27.20 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [ ]:
# Fit the model
history = model.fit(
    traindata,
    validation_data=devdata,
    epochs=30,
    # batch_size=64  # Specify your desired batch size here
)


Epoch 1/30
457/953 ━━━━━━━━━━━━━━━━━━━━ 16:21 2s/step - accuracy: 0.4746 - loss: 1.0591

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())

In [ ]:
plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
    
plt.subplot(222)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the ImageDataGenerator for testing (with normalization only)
testingData = ImageDataGenerator(
    rescale=1.0/255.0  # Normalize pixel values to [0, 1]
)

# Load your test data
testdata = testingData.flow_from_directory(
    directory="data/test",
    target_size=(224, 224),  # Resize images to the desired dimensions
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=CLASSES  # Specify the class names
)


In [ ]:
modelName = "MyApplesAndBanans.keras";
model.save(modelName);

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load your pre-trained model
model = tf.keras.models.load_model(modelName)

# Create the test data generator
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Specify the path to your test set
test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(IMAGESIZE, IMAGESIZE),  # Change this to the input size of your model
    class_mode='categorical',  # or 'binary' depending on your model
    shuffle=False,
    classes = CLASSES
)

# Evaluate the model using the generator
loss, accuracy = model.evaluate(test_generator, verbose=1)
print(f"Accuracy: {accuracy}")

# Get predictions
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Get true labels
y_true = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix")
print(cm)

# Calculate additional metrics
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
error_rate = 1 - accuracy

print(f"Precision: {precision}")
print(f"Sensitivity (Recall): {recall}")
print(f"Error Rate: {error_rate}")

# Classification Report
print("Classification Report")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
